In [1]:
!pip install -e 'git+https://github.com/mjkvaak/ImageDataAugmentor'
!pip install -U albumentations
!pip install kaggle
!pip install pandas
!pip install sklearn
!pip install tensorflow
!pip install scikit-image
!pip install keras
!pip install seaborn 
!pip install matplotlib
# RUN apt-get update && apt-get install -y python3-opencv
# RUN pip install opencv-python
!pip install opencv-python
!wand

ERROR: Could not detect requirement name for 'git+https://github.com/mjkvaak/ImageDataAugmentor', please specify one with #egg=your_package_name
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 102 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 19.2 MB/s eta 0:00:01
     |████████████████████████████████| 39.3 MB 27.0 MB/s eta 0:00:01    |██████▏                         | 7.5 MB 27.0 MB/s eta 0:00:02     |█████████████████████████▏      | 30.9 MB 27.0 MB/s eta 0:00:01     |███████████████████████████▋    | 33.9 MB 27.0 MB/s eta 0:00:01
     |████████████████████████████████| 30.2 MB 27.5 MB/s eta 0:00:01     |██████████████████████▍         | 21.2 MB 27.5 MB/s eta 0:00:01
     |████████████████████████████████| 47.6 MB 401 kB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 29.3 MB/s eta 0:00:01
 

In [1]:
import tensorflow as tf 
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
#from wand.image import Image

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
# my output was => ['/device:CPU:0']
# good output must be => ['/device:CPU:0', '/device:GPU:0']


['/device:CPU:0', '/device:GPU:0']


In [5]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1528776364144594504
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1383149979
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7836271992578610828
 physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
 xla_global_id: 416903419]

In [18]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('CPU')))

Num GPUs Available:  1


In [20]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], False)

RuntimeError: Physical devices cannot be modified after being initialized

In [4]:
!mkdir -p /root/.kaggle
!cp kaggle.json /root/.kaggle
!ls /root/.kaggle

kaggle.json


In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [2]:
import shutil

total, used, free = shutil.disk_usage("/")

print("Total: %d GiB" % (total // (2**30)))
print("Used: %d GiB" % (used // (2**30)))
print("Free: %d GiB" % (free // (2**30)))

Total: 476 GiB
Used: 129 GiB
Free: 347 GiB


In [7]:
!kaggle datasets download -d nurmukhammed7/idrd-train-set

100%|█████████████████████████████████████▉| 27.1G/27.1G [09:48<00:00, 54.3MB/s]
100%|██████████████████████████████████████| 27.1G/27.1G [09:48<00:00, 49.3MB/s]


In [8]:
!unzip -q idrd-train-set.zip -d idrd-train-set

In [8]:
# PATH = '2020-11-30b_resnext50_32x4d.pth'

# model = torch.load(PATH, map_location='cpu')


In [3]:
data_train = r"E:\LivenessDetectionDatasets\IDR&D\data"
labels = ['2dmask', 'printed', 'real', 'replay']
image_size = 160
#"C:\Users\user\Desktop\Liveness Detection\JUPYTER NOTEBOOKS\Ternaus works\dataset\data\2dmask\2dmask_2\2dmask_2_01.png"

In [4]:
X = []
y = []


def image_reader(data_train):
    for label in labels:
        join_label = os.path.join(data_train, label)
        for label_folder in tqdm(os.listdir(join_label)):
            join_folder = os.path.join(join_label, label_folder)
            for folder_image in os.listdir(join_folder):
                try:
                    img_path = os.path.join(join_folder, folder_image)
                    image = cv2.imread(img_path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (image_size, image_size))
                    X.append(image)
                    y.append(labels.index(label))
                except: 
                    pass
    return X, y

X, y = image_reader(data_train)

100%|██████████████████████████████████████████████████████████████████████████████| 4014/4014 [05:34<00:00, 11.99it/s]


In [5]:
X = np.array(X)
y = np.array(y)

In [6]:
print(X.shape, y.shape)

(57655, 160, 160, 3) (57655,)


In [7]:
import pandas as pd
pd.DataFrame(y).value_counts()

1    25180
3    20070
2    10240
0     2165
dtype: int64

### Model Preparation

In [18]:
from albumentations import SmallestMaxSize, RandomResizedCrop, ColorJitter, Blur, Normalize, CenterCrop, Compose
import albumentations

In [24]:
from ImageDataAugmentor.image_data_augmentor import *

ModuleNotFoundError: No module named 'ImageDataAugmentor'

In [14]:
from tensorflow.keras.layers import Dense,Dropout,Input,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import model_from_json
import json

In [15]:
train_aug = Compose([
    SmallestMaxSize(always_apply=False, max_size=256, p=1),
    CenterCrop(always_apply =False, height = 120, width = 120, p=1),
    RandomResizedCrop(always_apply = False, height = 120, width = 120, p=1),
    ColorJitter(always_apply = False, brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.2, p=1),
    Blur(always_apply = False, p=0.5),
    Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225),always_apply = False, max_pixel_value = 255.0)],)

# val_aug = Compose([
#     SmallestMaxSize(always_apply = False, max_size = 256, p=1),
#     CenterCrop(always_apply =False, height = 120, width = 120, p=1),
#     Normalize(mean=(0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225) ,always_apply = False, max_pixel_value = 255., p =1)
# ])

NameError: name 'Compose' is not defined

In [24]:
datagen = ImageDataAugmentor(
    featurewise_center=True,
    featurewise_std_normalization = True,
    augment=train_aug,
    validation_split= 0.1
    )

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1235545)
num_classes = len(np.unique(y))

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


In [17]:
train_datagen = ImageDataGenerator(brightness_range=(0.8,1.2),rotation_range=30,width_shift_range=0.2,
                                   height_shift_range=0.2,fill_mode='nearest',shear_range=0.2,zoom_range=0.3,
                                   validation_split=0.15)
valid_datagen = ImageDataGenerator(validation_split=0.15)

In [18]:
train_generator = train_datagen.flow(x_train, y_train, batch_size=32,shuffle=True,  subset='training')

In [19]:
valid_generator = valid_datagen.flow(x_train, y_train,batch_size=25,shuffle=True,  subset='validation')

In [20]:
# compute quantities required for featurewise normalization
train_generator = datagen.flow(x_train, y_train, batch_size=32, subset='training')
validation_generator = datagen.flow(x_train, y_train, batch_size=32, subset='validation')

NameError: name 'datagen' is not defined

In [21]:
from tensorflow.keras.layers import Dense,Dropout,Input,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import model_from_json
import json

In [22]:
from tensorflow.keras.applications.resnet50 import ResNet50
base = ResNet50(include_top = False, weights = 'imagenet', input_shape = (image_size,image_size,3))

2021-11-23 14:09:18.377704: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-23 14:09:19.857657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22307 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
2021-11-23 14:09:19.860923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22307 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:02:00.0, compute capability: 8.6


94781440/94765736 [==============================] - 2s 0us/step


In [24]:
base.trainable = False

In [61]:
import visualkeras 
visualkeras.layered_view(base)

ModuleNotFoundError: No module named 'visualkeras'

In [25]:
from tensorflow.keras.layers import *

x = GlobalAveragePooling2D()(base.output)
x = Dense(2048, activation = 'relu')(x)
x = Dropout(0.5)(x)
output = Dense(4, activation = 'softmax')(x)

model  = Model(inputs = base.input, outputs = output)



In [26]:
lr = 1e-3

model.compile(loss='categorical_crossentropy', optimizer = Adam(lr = lr), metrics = ['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [27]:
checkpoint = ModelCheckpoint('WeightsResNet50 IDRND.h5', monitor = 'val_loss', save_best_only = True, mode = 'auto', verbose= 1)

In [ ]:
history = model.fit_generator(
    train_generator,
    validation_data = valid_generator, 
    epochs = 100,
    callbacks=[checkpoint])

/tmp/ipykernel_67/1069951509.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/100


2021-11-23 14:11:41.886179: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100


   1/1226 [..............................] - ETA: 2:35:34 - loss: 2.5079 - accuracy: 0.2188

2021-11-23 14:11:45.397951: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1226/1226 [==============================] - ETA: 0s - loss: 0.5270 - accuracy: 0.8124
Epoch 00001: val_loss improved from inf to 0.29316, saving model to WeightsResNet50 IDRND.h5


/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


1226/1226 [==============================] - 253s 200ms/step - loss: 0.5270 - accuracy: 0.8124 - val_loss: 0.2932 - val_accuracy: 0.8818
Epoch 2/100
1226/1226 [==============================] - ETA: 0s - loss: 0.3746 - accuracy: 0.8528
Epoch 00002: val_loss improved from 0.29316 to 0.27541, saving model to WeightsResNet50 IDRND.h5
1226/1226 [==============================] - 244s 199ms/step - loss: 0.3746 - accuracy: 0.8528 - val_loss: 0.2754 - val_accuracy: 0.8884
Epoch 3/100
1226/1226 [==============================] - ETA: 0s - loss: 0.3483 - accuracy: 0.8641
Epoch 00003: val_loss improved from 0.27541 to 0.23019, saving model to WeightsResNet50 IDRND.h5
1226/1226 [==============================] - 242s 197ms/step - loss: 0.3483 - accuracy: 0.8641 - val_loss: 0.2302 - val_accuracy: 0.9107
Epoch 4/100
1226/1226 [==============================] - ETA: 0s - loss: 0.3302 - accuracy: 0.8719
Epoch 00004: val_loss improved from 0.23019 to 0.19103, saving model to WeightsResNet50 IDRND.h5
1

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("ModelResNet50_IDRND.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
import matplotlib.pyplot as plt
train_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']
epochs = range(1,101)
plt.plot(epochs,train_accuracy,'g',label='Training Accuracy')
plt.plot(epochs,validation_accuracy,'b',label='Validation Accuracy')
plt.title('Training Vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,101)
plt.plot(epochs,train_loss,'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [9]:
from tensorflow.keras.models import load_model

In [10]:
model = load_model('WeightsResNet50 IDRND(2).h5')